### 注意一些细节

In [3]:
import torch
import torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter

from self_model import * 

In [4]:
train_data=torchvision.datasets.CIFAR10(root='../data',train=True,transform=torchvision.transforms.ToTensor(),download=True)
test_data=torchvision.datasets.CIFAR10(root='../data',train=False,transform=torchvision.transforms.ToTensor(),download=True)
print('训练集数据量大小为{}'.format(len(train_data)))
print('测试集数据量大小为{}'.format(len(test_data)))

100%|██████████| 170M/170M [01:22<00:00, 2.06MB/s] 


In [45]:
from torch.utils.data import DataLoader

train_dataloader=DataLoader(train_data,batch_size=64,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=64,shuffle=True)

In [46]:
net=Net()
##损失函数
loss_func=nn.CrossEntropyLoss()
##优化器
learning_rate=1e-2
optm=torch.optim.SGD(net.parameters(),lr=learning_rate,momentum=0.9)

In [47]:
###设置训练时的一些参数
total_train_step=0
total_test_step=0
##训练的轮数
epoch=10
for i in range(epoch):
    print('-------------------第{}轮训练开始-------------------'.format(i))
    for data in train_dataloader:
        imgs,label=data
        optm.zero_grad()
        out_put=net.forward(imgs)
        loss=loss_func(out_put,label)
        loss.backward()
        optm.step()
        
        if total_train_step%100==0:
            print('训练次数{},损失值{}'.format(total_train_step,loss.item()))
        total_train_step+=1
    
    with torch.no_grad():
        total_test_loss=0
        total_tes_acc=0
        for data in test_dataloader:
            imgs,label=data
            out_put=net.forward(imgs)
            loss=loss_func(out_put,label)
            total_test_loss+=loss.item()
            total_test_step+=1
            total_tes_acc+=(out_put.argmax(dim=1)==label).sum()
        print('整体数据损失{}'.format(total_test_loss))
        print('整体数据正确率{}'.format(total_tes_acc/len(test_data)))
        total_test_step+=1
    torch.save(net.state_dict(),'model_{}.pth'.format(i))
    print('模型已保存{}'.format(total_test_loss))
####  可以使用tensorboard来可视化训练过程中数据，和训练效果

-------------------第0轮训练开始-------------------
训练次数0,损失值2.3045668601989746
训练次数100,损失值1.8871444463729858
训练次数200,损失值1.7419120073318481
训练次数300,损失值1.6242985725402832
训练次数400,损失值1.5689527988433838
训练次数500,损失值1.5177866220474243
训练次数600,损失值1.390172004699707
训练次数700,损失值1.680038571357727
整体数据损失208.696240067482
整体数据正确率0.5342000126838684
模型已保存208.696240067482
-------------------第1轮训练开始-------------------
训练次数800,损失值1.193179726600647
训练次数900,损失值1.2019984722137451
训练次数1000,损失值1.2389649152755737
训练次数1100,损失值1.1770185232162476
训练次数1200,损失值1.1630011796951294
训练次数1300,损失值0.9772892594337463
训练次数1400,损失值1.1520715951919556
训练次数1500,损失值1.1956537961959839


KeyboardInterrupt: 

### 模型.train() 和模型.eval() 这两个方法实际上是针对特定的层才有意义，例如dropout 和bn层，当train时就开始训练，eval的时候则使用原始模型执行